# Plan
1. Create ML Model based on this dataset
2. Create models for specific industries 

# Project Structure:
1. Abstract
2. Load data
3. EDA
4. Preprocess data
5. Feature engineering
6. Train, test split
7. Modelling
8. Model evaluation
9. Hyperparameter tuning 
10. Model Interpretation 
11. Results and conclusions
12. References and Acknowledgments

# Abstract 
Title: Predicting Scope 3 Emissions using Machine Learning: A Novel Approach

The following research of mine is based on a study conducted by Serafeim, George and Velez Caicedo, Gladys. 2022. "Machine Learning Models for Prediction of Scope 3 Carbon Emissions." Harvard Business School Working Paper, 2022. 

I would like to thank the authors for sharing their methodology and data, which allows me to independently conduct research and modeling and then compare the results with the conclusions of the researchers thanks to which I have a great opportunity to learn and to lead the research in new directions by updating the model.

In [1]:
import polars as pl
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport

## Data Guidence

Copyright © 2022 by George Serafeim and Gladys Velez Caicedo. "Machine Learning Models for Prediction of Scope 3 Carbon Emissions." Harvard Business School Working Paper, 2022		
Funding for this research was provided in part by Harvard Business School.		
		
Data source: 		
“Serafeim, George and Velez Caicedo, Gladys. 2022. "Machine Learning Models for Prediction of Scope 3 Carbon Emissions." Harvard Business School Working Paper, 2022		
		
GUIDANCE:		
Column A "Year" is the year in which the environmental impact was incurred by the firm's operations.		
		
Column B "Company Name" is the name of the issuer.		
		
Column C "Country" is the country in which the companies' headquarters are located.		
		
Column D "Industry" refers to the Exiobase industry category to which the firm belongs: We provide only the Exiobase industries here as they are open source, but in our paper, we use GICS taxonomy as fixed effects.		
All Exiobase industries are based on the International Standard Industrial Classification Revision 3.1 (ISIC). 	To learn more about ISIC and a comprehensive list of industries included, please refer to: unstats.un.org/unsd/statcom/doc02/isic.pdf	
	For example, the term "nec" refers to "not elsewhere classified."	
		
Column E "GHG Intensity (Sales)" is the monetized GHG impact of the firm's operations during the specific year indicated in column A divided by revenue in that year		
		
Column F "GHG Intensity (Op Inc)" is the monetized GHG impact of the firm's operations during the specific year indicated in column A divided by operating income in that year		
		
Column G "Total GHG Environmental Cost (Scope 1, 2, 3) " is the total monetized GHG environmental impact of Scope 1, 2, and 3 emissions of the firm's operations during the specific year indicated in Column A.		
		
Columns H-J are Scope 1, 2, 3 Emissions		
Each scope of emissions is defined by the GHG Protocol. More information can be found at the Greenhouse Gas Protocol: https://ghgprotocol.org/		
	Column H:	 Scope 1 Emissions: emissions from direct operations that occur from sources that are controlled or owned by the firm 
	Column I:	 Scope 2 Emissions: emissions associated with the purchase of electricity, steam, heat, or cooling as a result of the firm's energy use 
	Column J:	 Scope 3 Emissions: emissions from 15 categories that are result of activities from assets not owned or controlled by the reporting firm, not within a firm's scope 1 and 2 boundary and occur through the value chain. 
		
Columns K-BC are fiveteen Scope 3 emissions category types in alphabetical order followed by an indicator variable denoting if the data point is company reported (0) or if the data point is predicted via machine learning (1)		
	Column K	Business Travel
	Column N:	Capital Goods
	Column Q:	Downstream Leased Assets
	Column T:	Downstream Transportation and Distribution
	Column W:	Employee Commuting
	Column Z:	End of Life Treatment of Sold Products
	Column AC:	Franchises
	Column AF:	Fuel-and-energy-related activities (not included in Scope 1 or 2)
	Column AI:	Investments
	Column AL:	Processing of Sold Products
	Column AO:	Purchased Goods and Services
	Column AR:	Upstream Leased Assets
	Column AU:	Upstream Transportation and Distribution
	Column AX:	Use of Sold Products
	Column BA:	Waste Generated in Operations
		
The dataset is a combination of primary firm reported emissions data supplemented with Scope 3 predictions by category.		
Our methodology takes firm reported values first and incorporates imputations only when companies' self-reported emissions data are not publicly available.		
If the data point is imputed, the Scope 3 category "Imputed" value is 1.		
If the data point is company reported, the Scope 3 category "Imputed" value is 0.		
The Scope 3 category "Test" column indicates if the data point was used to "train" or "test" the machine learning model. If no company value is reported, the value. Is set to "none". 		
		
Other Notes:		
The "Final Raw Sample(0%)" tab includes all raw outputs, discounted at 0%, from our environmental impact calculation methodology. The Social Cost of Carbon discounted at 0% applied here is roughly $300 USD per metric ton of emissions.		
The "Final Raw Sample(3%)" tab includes all raw outputs, discounted at 3%, from our environmental impact calculation methodology. The Social Cost of Carbon discounted at 3% applied here is roughly $100 USD per metric ton of emissions.		
All observations in the tabs are sorted by 1) Year in descending order, 2) Industry in alphabetical order, and 3) Environmental Intensity (Sales) in descending order.		
		
		
Also, if you are a researcher planning to use the data in an academic research project, please email us and we will send you a file including ISINs to facilitate merging with other datasets.		
Our team can be reached at: ImpactWeightedAccounts@hbs.edu

# Load data 

In [3]:
# Trainging dataset
df = pl.read_excel(
    "/home/adamprzychodni/Documents/GitHub/scope-3-emissions-prediction/data/01_raw/IWA-External-Scope-3-Data.xlsx",
    sheet_name="3%",
)

# 3% tab includes all raw outputs, discounted at 3%, from our environmental impact calculation methodology. 
# The Social Cost of Carbon discounted at 3% applied here is roughly $100 USD per metric ton of emissions.

Year,Company Name,Country,Industry (Exiobase),GHG Intensity (Sales),GHG Intensity (Op Income),"Total GHG Environmental Impact (Scope 1, 2, 3)",Scope 1,Scope 2,Scope 3,Business Travel,Business Travel Set,Business Travel Imputed,Capital Goods,Capital Goods Set,Capital Goods Imputed,Downstream Leased Assets,Downstream Leased Assets Set,Downstream Leased Assets Imputed,Downstream Transportation and Distribution,Downstream Transportation and Distribution Set,Downstream Transportation and Distribution Imputed,Employee Commuting,Employee Commuting Set,Employee Commuting Imputed,End of Life Treatment of Sold Products,End of Life Treatment of Sold Products Set,End of Life Treatment of Sold Products Imputed,Franchises,Franchises Set,Franchises Imputed,Fuel-and-energy-related activities (not included in Scope 1 or 2),Fuel-and-energy-related activities (not included in Scope 1 or 2) Set,Fuel-and-energy-related activities (not included in Scope 1 or 2) Imputed,Investments,Investments Set,Investments Imputed,Processing of Sold Products,Processing of Sold Products Set,Processing of Sold Products Imputed,Purchased Goods and Services,Purchased Goods and Services Set,Purchased Goods and Services Imputed,Upstream Leased Assets,Upstream Leased Assets Set,Upstream Leased Assets Imputed,Upstream Transportation and Distribution,Upstream Transportation and Distribution Set,Upstream Transportation and Distribution Imputed,Use of Sold Products,Use of Sold Products Set,Use of Sold Products Imputed,Waste Generated in Operations,Waste Generated in Operations Set,Waste Generated in Operations Imputed
i64,str,str,str,f64,f64,f64,f64,f64,f64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64
2020,"""REA GROUP LTD""","""AUSTRALIA""","""Recreational, …",-0.008351,-0.017857,-5.0822e6,-79.675141,-109280.146543,-4.9728e6,-249656.362284,"""train""",0,-47577.441157,"""predicted""",1,-34431.042943,"""predicted""",1,-29981.755443,"""predicted""",1,-72788.932106,"""train""",0,-140569.71251,"""predicted""",1,0.0,"""predicted""",1,-10050.44989,"""train""",0,0.0,"""predicted""",1,-3.8007e6,"""predicted""",1,-40327.003354,"""train""",0,-115870.418895,"""train""",0,-162293.708729,"""predicted""",1,-251742.712753,"""predicted""",1,-16800.072524,"""train""",0
2020,"""SEEK LTD""","""AUSTRALIA""","""Recreational, …",-0.006768,-0.041618,-7.4505e6,-417435.789792,-129756.657702,-6.9033e6,-312212.729891,"""test""",0,-35057.061905,"""train""",0,-110065.515787,"""predicted""",1,-556075.57123,"""predicted""",1,-42569.289457,"""train""",0,-126000.543926,"""predicted""",1,0.0,"""predicted""",1,-12065.092734,"""test""",0,0.0,"""predicted""",1,-788442.427983,"""predicted""",1,-98114.244683,"""train""",0,-135220.095921,"""train""",0,-383578.891627,"""predicted""",1,-4.2188e6,"""predicted""",1,-85138.578913,"""train""",0
2020,"""SIMS LTD""","""UNITED STATES""","""Manufacture of…",-0.19019,null,-6.4545e8,-7.2562e6,-7.5715e6,-6.3062e8,-236193.539939,"""predicted""",1,-3.1044e6,"""predicted""",1,-688393.215597,"""predicted""",1,-1.3227e7,"""train""",0,-464114.523841,"""predicted""",1,-383465.069998,"""predicted""",1,0.0,"""predicted""",1,-2.2353e6,"""predicted""",1,0.0,"""predicted""",1,-1.0371e8,"""predicted""",1,-3.3176e7,"""predicted""",1,-164244.611459,"""predicted""",1,-4.4390e6,"""predicted""",1,-3.8444e8,"""predicted""",1,-8.4353e7,"""train""",0
2020,"""TRANSURBAN GRO…","""AUSTRALIA""","""Other land tra…",-0.039985,-0.218081,-9.9784e7,-499790.775411,-1.5853e7,-8.3431e7,-291611.014941,"""train""",0,-4.6137e7,"""train""",0,-403542.067237,"""predicted""",1,-341464.888689,"""predicted""",1,-270212.548583,"""train""",0,-126000.543926,"""predicted""",1,0.0,"""predicted""",1,-1.9816e6,"""train""",0,-5.2981e6,"""test""",0,-8.7355e6,"""predicted""",1,-1.8394e7,"""train""",0,-223065.353185,"""predicted""",1,-920795.357055,"""predicted""",1,-52758.601735,"""predicted"""

In [4]:
# Dataset with 0% rate

df_0_percent = pl.read_excel(
    "/home/adamprzychodni/Documents/GitHub/scope-3-emissions-prediction/data/01_raw/IWA-External-Scope-3-Data.xlsx",
    sheet_name="0%",
)

# 0% tab includes all raw outputs, discounted at 0%, from our environmental impact calculation methodology. 
# The Social Cost of Carbon discounted at 0% applied here is roughly $300 USD per metric ton of emissions

Year,Company Name,Country,Industry (Exiobase),GHG Intensity (Sales),GHG Intensity (Op Income),"Total GHG Environmental Impact (Scope 1, 2, 3)",Scope 1,Scope 2,Scope 3,Business Travel,Business Travel Set,Business Travel Imputed,Capital Goods,Capital Goods Set,Capital Goods Imputed,Downstream Leased Assets,Downstream Leased Assets Set,Downstream Leased Assets Imputed,Downstream Transportation and Distribution,Downstream Transportation and Distribution Set,Downstream Transportation and Distribution Imputed,Employee Commuting,Employee Commuting Set,Employee Commuting Imputed,End of Life Treatment of Sold Products,End of Life Treatment of Sold Products Set,End of Life Treatment of Sold Products Imputed,Franchises,Franchises Set,Franchises Imputed,Fuel-and-energy-related activities (not included in Scope 1 or 2),Fuel-and-energy-related activities (not included in Scope 1 or 2) Set,Fuel-and-energy-related activities (not included in Scope 1 or 2) Imputed,Investments,Investments Set,Investments Imputed,Processing of Sold Products,Processing of Sold Products Set,Processing of Sold Products Imputed,Purchased Goods and Services,Purchased Goods and Services Set,Purchased Goods and Services Imputed,Upstream Leased Assets,Upstream Leased Assets Set,Upstream Leased Assets Imputed,Upstream Transportation and Distribution,Upstream Transportation and Distribution Set,Upstream Transportation and Distribution Imputed,Use of Sold Products,Use of Sold Products Set,Use of Sold Products Imputed,Waste Generated in Operations,Waste Generated in Operations Set,Waste Generated in Operations Imputed
i64,str,str,str,f64,f64,f64,f64,f64,f64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64,f64,str,i64
2020,"""REA GROUP LTD""","""AUSTRALIA""","""Recreational, …",-0.023173,-0.049552,-1.4103e7,-221.094759,-303247.25463,-1.3799e7,-692784.635251,"""train""",0,-132025.156166,"""predicted""",1,-95544.52091,"""predicted""",1,-83197.957861,"""predicted""",1,-201985.854948,"""train""",0,-390074.325036,"""predicted""",1,0.0,"""predicted""",1,-27889.524616,"""train""",0,0.0,"""predicted""",1,-1.0547e7,"""predicted""",1,-111905.533085,"""train""",0,-321534.949706,"""train""",0,-450357.390362,"""predicted""",1,-698574.159444,"""predicted""",1,-46619.409211,"""train""",0
2020,"""SEEK LTD""","""AUSTRALIA""","""Recreational, …",-0.01878,-0.115489,-2.0675e7,-1.1584e6,-360068.607726,-1.9156e7,-866375.606134,"""test""",0,-97281.694017,"""train""",0,-305426.617255,"""predicted""",1,-1.5431e6,"""predicted""",1,-118127.771307,"""train""",0,-349645.569081,"""predicted""",1,0.0,"""predicted""",1,-33480.063525,"""test""",0,0.0,"""predicted""",1,-2.1879e6,"""predicted""",1,-272262.403386,"""train""",0,-375229.391209,"""train""",0,-1.0644e6,"""predicted""",1,-1.1707e7,"""predicted""",1,-236255.542613,"""train""",0
2020,"""SIMS LTD""","""UNITED STATES""","""Manufacture of…",-0.527767,null,-1.7911e9,-2.0136e7,-2.1010e7,-1.7499e9,-655425.937951,"""predicted""",1,-8.6146e6,"""predicted""",1,-1.9103e6,"""predicted""",1,-3.6705e7,"""train""",0,-1.2879e6,"""predicted""",1,-1.0641e6,"""predicted""",1,0.0,"""predicted""",1,-6.2030e6,"""predicted""",1,0.0,"""predicted""",1,-2.8780e8,"""predicted""",1,-9.2062e7,"""predicted""",1,-455771.053464,"""predicted""",1,-1.2318e7,"""predicted""",1,-1.0668e9,"""predicted""",1,-2.3408e8,"""train""",0
2020,"""TRANSURBAN GRO…","""AUSTRALIA""","""Other land tra…",-0.110958,-0.605164,-2.7690e8,-1.3869e6,-4.3991e7,-2.3152e8,-809206.818416,"""train""",0,-1.2803e8,"""train""",0,-1.1198e6,"""predicted""",1,-947548.9677,"""predicted""",1,-749827.083107,"""train""",0,-349645.569081,"""predicted""",1,0.0,"""predicted""",1,-5.4989e6,"""train""",0,-1.4702e7,"""test""",0,-2.4241e7,"""predicted""",1,-5.1044e7,"""train""",0,-618995.83864,"""predicted""",1,-2.5552e6,"""predicted""",1,-146402.632503,"""predicted""",1,-707819.078872,"""test""",0
2020,"""WORLE

# EDA

1. Data profiling
2. 

In [8]:
# load training dataset as pandas dataframe to use ydata_profiling
df_eda = pd.read_excel("/home/adamprzychodni/Documents/GitHub/scope-3-emissions-prediction/data/01_raw/IWA-External-Scope-3-Data.xlsx", sheet_name="3%")

In [11]:
def data_profiling(df: pd.DataFrame, name: str="data_profiling_report", interface: str="html") -> None:
  """
  This function generates a data profiling report using the pandas_profiling package.
  
  Args:
      df (pd.DataFrame): The DataFrame to profile.
      name (str, optional): The title of the profile report. Defaults to "data_profiling_report".
      interface (str, optional): The format of the report. Defaults to "html".
                                  Choose between 'html' or 'widget'.
  
  Raises:
      ValueError: If df is not a pandas DataFrame or name is not a string or 
                  if interface is not 'html' or 'widget'
  """
  
  # Check if df is a pandas DataFrame
  if not isinstance(df, pd.DataFrame):
    raise ValueError("df should be a pandas DataFrame")
    
  # Check if name is a string
  if not isinstance(name, str):
    raise ValueError("name should be a string")
  
  # Check if interface is a string and a valid option
  if not isinstance(interface, str) or interface not in ['html', 'widget']:
    raise ValueError("interface should be a string, either 'html' or 'widget'")
  
  profile = ProfileReport(df, title=name, explorative=True)

  if interface == "html":
    profile.to_file(f"{name}.html")
    logging.info(f"Report {name} generated in html format, check files.")
    
  elif interface == "widget":
    logging.info(f"Report {name} will be generated as a widget, it might take a while.")
    profile.to_widgets()


In [ ]:
# Raw data profiling
data_profiling(df_eda)

# Feature engineering

# Train, test split

# Modelling

# Model evaluation

# Hyperparameter tuning

# Model Interpretation

# Results and conclusions

# References and Acknowledgments